<center><h1>Recommender System Basic</h1></center>
<hr>

## References

> **Basic Knowledge**<br>
> [Basics about Recommendation](https://datascienceschool.net/view-notebook/fcd3550f11ac4537acec8d18136f2066/)<br>
> 
> **pandas**<br>
> [pandas: sort DataFrame](https://rfriend.tistory.com/281)<br>
> [pandas: filter value by list -> isin()](https://stackoverflow.com/questions/45803676/python-pandas-loc-filter-for-list-of-values)<br>
> [pandas: append the DataFrame -> append()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html)<br>
> [pandas: to pivot table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html)<br>
> [pandas: pivot_table vs pivot and its Possible Errors](https://rfriend.tistory.com/275)<br>
> [pandas: duplication processing](https://appdividend.com/2020/03/07/python-pandas-find-duplicate-rows-in-dataframe-based-on-all-or-selected-columns/)<br>
> [pandas: remove duplicates](https://www.geeksforgeeks.org/python-pandas-dataframe-drop_duplicates/)<br>
> [pandas: list to DataFrame in row](https://stackoverflow.com/questions/42202872/how-to-convert-list-to-row-dataframe-with-pandas)<br>
>
> **numpy**<br>
> [numpy: random number selection -> choices()](https://datascienceschool.net/view-notebook/8bf41f87a08b4c44b307799577736a28/)<br>
> 
> **sci-kit learn**<br>
> [sci-kit learn: calculate pairwise cosine similarity](https://stackoverflow.com/questions/45387476/cosine-similarity-between-each-row-in-a-dataframe-in-python)<br>


## Table of Contents <a id="top"></a>

1. [Prepare Dataset](#1)
2. [Train Test Split](#2)
3. [Data Modeling](#3)
    1. [K-NN Based](#3.1)

<hr>

# 1. Prepare Dataset <a id="1"></a>
<p style="text-align:right;"><a href="#top">top 🔝</a></p>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
playlist_df = pd.read_csv('./data/mpd.slice.0-999.playlist.csv', encoding='utf-8')
track_df = pd.read_csv('./data/mpd.slice.0-999.track.csv', encoding='utf-8')
playlist_track_df = pd.read_csv('./data/mpd.slice.0-999.playlist.track.csv', encoding='utf-8')

In [3]:
playlist_df

,pid,name,num_tracks,num_artists,num_albums,num_followers,num_edits,collaborative,duration_ms,modified_at,description
0,0,Throwbacks,52,37,47,1,6,False,11532414,1493424000,NaN
1,1,Awesome Playlist,39,21,23,1,5,False,11656470,1506556800,NaN
2,2,korean,64,31,51,1,18,False,14039958,1505692800,NaN
3,3,mat,126,86,107,1,4,False,28926058,1501027200,NaN
4,4,90s,17,16,16,2,7,False,4335282,1401667200,NaN
...,...,...,...,...,...,...,...,...,...,...,...
995,995,old,41,36,40,1,8,False,9917901,1507852800,NaN
996,996,Daze,17,15,17,1,13,False,3699248,1479254400,NaN
997,997,rap,119,82,98,1,63,False,27538723,1410307200,NaN
998,998,Country,108,40,75,1,37,False,24950143,1507939200,NaN


In [4]:
track_df

,tid,track_name,track_uri,artist_name,artist_uri,album_name,album_uri,duration_ms
0,0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863
1,1,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800
2,2,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933
3,3,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266
4,4,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600
...,...,...,...,...,...,...,...,...
34438,34438,I Don't Know,spotify:track:3uCHI1gfOUL5j5swEh0TcH,Jon D,spotify:artist:5HCypjplgh5uQezvBpOfXN,Roots,spotify:album:2KEQtuVl1cYsTYtVRUrNVi,189183
34439,34439,The Answer,spotify:track:0P1oO2gREMYUCoOkzYAyFu,Big Words,spotify:artist:0sHN89qak07mnug3LVVjzP,"Hollywood, a Beautiful Coincidence",spotify:album:5jrsRHRAmetu5e7RRBoxj7,263679
34440,34440,25.22,spotify:track:2oM4BuruDnEvk59IvIXCwn,Allan Rayman,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,Roadhouse 01,spotify:album:3CbNgBzI7r9o0F6VjH9sTY,189213
34441,34441,Good Feeling,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,Jon Jason,spotify:artist:77bNdkKYBBmc30CisCA6tE,Good Feeling,spotify:album:2dZ7oVNQBeLlpoUYfbEsJP,194720


In [5]:
playlist_track_df

,pid,tid,pos
0,0,0,0
1,0,1,1
2,0,2,2
3,0,3,3
4,0,4,4
...,...,...,...
67498,999,2076,39
67499,999,34439,40
67500,999,34440,41
67501,999,34441,42


In [6]:
def load_data(raw, min_track_freq=25, min_playlist_freq=10):
    dataset = raw.copy()
    dataset = dataset.rename(columns={'pos': 'have'})
    dataset['have'] = 1
    
    tid_count = dataset['tid'].value_counts()
    tid_index = tid_count.loc[tid_count.values >= min_track_freq].index
    dataset = dataset.loc[dataset['tid'].isin(tid_index)]
    
    pid_count = dataset['pid'].value_counts()
    pid_index = pid_count.loc[pid_count.values >= min_playlist_freq].index
    dataset = dataset.loc[dataset['pid'].isin(pid_index)]
    
    return dataset.sort_values(by=['pid'])  


In [7]:
data = load_data(playlist_track_df)

In [8]:
data

,pid,tid,have
1374,28,525,1
1378,28,1331,1
1379,28,1332,1
1380,28,1333,1
1381,28,1334,1
...,...,...,...
66348,984,2700,1
66320,984,536,1
66406,984,1335,1
66354,984,4796,1


In [9]:
def remove_duplicated(dataset):
    dataset = dataset.copy()
    dataset = dataset.drop_duplicates()
    return dataset

In [10]:
data = remove_duplicated(data)

# 2. Train Test Split <a id="2"></a>
<p style="text-align:right;"><a href="#top">top 🔝</a></p>

In [11]:
def train_test(data, rate=0.2):
    train = data.copy()
    test = pd.DataFrame(columns=['pid', 'tid', 'have'])
    
    for pid in train['pid'].unique():
        index = train.loc[train['pid'] == pid].index
        if len(index) != len(set(index)):
            print(pid, index)
        length = len(index)
        sample = index[np.random.choice(length, int(rate * length), replace=False)]
        for s in sample:
            test = test.append(train.loc[s])
            train = train.drop(s)
    return train, test

In [12]:
train, test = train_test(data)

In [13]:
test

,pid,tid,have
1374,28,525,1
1380,28,1333,1
2645,42,1828,1
2558,42,30,1
3038,53,2712,1
...,...,...,...
65998,980,1333,1
66000,980,2715,1
65958,980,4837,1
66357,984,3928,1


# 3. Data Modeling <a id="3"></a>
<p style="text-align:right;"><a href="#top">top 🔝</a></p>

In [14]:
data_pivot = data.pivot(index='pid', columns='tid', values='have')
data_pivot = data_pivot.fillna(0.0)
data_pivot

tid,2,4,5,9,30,38,75,76,80,327,...,4796,4811,4832,4837,5687,5691,5979,6401,7053,7520
pid,,,,,,,,,,,,,,,,,,,,,
28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
train_pivot = data_pivot.copy()
for pid, tid in zip(test['pid'], test['tid']):
    train_pivot.loc[pid, tid] = 0.0
train_pivot

tid,2,4,5,9,30,38,75,76,80,327,...,4796,4811,4832,4837,5687,5691,5979,6401,7053,7520
pid,,,,,,,,,,,,,,,,,,,,,
28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## 3.1. K-NN Based <a id="3.1"></a>
<p style="text-align:right;"><a href="#top">top 🔝</a></p>

In [16]:
def mean_square_difference_similarity(pivot):
    msd_playlist_matrix = np.zeros((pivot.shape[0], pivot.shape[0]))
    for i, udx in enumerate(pivot.index):
        for j, vdx in enumerate(pivot.index):
            u = np.array(pivot.loc[udx])
            v = np.array(pivot.loc[vdx])
            msd = np.mean(np.square(u - v))
            msd_similarity = 1.0 / (msd + 1.0)
            msd_playlist_matrix[i, j] = msd_similarity
            
    msd_track_matrix = np.zeros((pivot.shape[1], pivot.shape[1]))
    for i, udx in enumerate(pivot.columns):
        for j, vdx in enumerate(pivot.columns):
            u = np.array(pivot.loc[:, udx])
            v = np.array(pivot.loc[:, vdx])
            msd = np.mean(np.square(u - v))
            msd_similarity = 1.0 / (msd + 1.0)
            msd_track_matrix[i, j] = msd_similarity
    
    return msd_playlist_matrix, msd_track_matrix

In [17]:
msd_playlist_matrix, msd_track_matrix = mean_square_difference_similarity(train_pivot)

In [18]:
def find_neighbors(index, matrix, k):
    neighbor_index = np.argsort(matrix[index])[::-1][1:k+1]
    neighbor_score = matrix[index, neighbor_index]
    return [(idx, score) for idx, score in zip(neighbor_index, neighbor_score)]

In [19]:
def knn_basic(data, similarity_matrix, k=10):
    rating = np.zeros_like(data)
    
    for i, pdx in enumerate(data.index):
        vs = find_neighbors(i, similarity_matrix, k)
        
        scores = [score for _, score in vs]
        denominator = np.sum(scores)
        
        for j, tdx in enumerate(data.columns):
            numerator = 0.
            for vdx, vsim in vs:
                numerator += data.iloc[vdx, j] * vsim
            
            rating[i, j] = numerator / denominator
            
    return rating

In [20]:
msd_basic_rating = knn_basic(train_pivot, msd_playlist_matrix)

In [21]:
def recommend(data, rating, n=10):
    columns = ['pid']
    for idx in range(n):
        columns.append('recommend-{}'.format(idx + 1))
        
    recommend_df = pd.DataFrame(columns=columns)

    for pdx, pid in enumerate(data.index):
        recommendations = np.argsort(rating[pdx])[::-1]
        counter = 0
        idx = 0
        row = [pid]
        while counter < n:
            tdx = recommendations[idx]
            
            if data.iloc[pdx, tdx] == 0.0:
                tid = data.columns[tdx]
                row.append(tid)
                counter += 1
            idx += 1
        recommend_df = recommend_df.append(pd.DataFrame([row], columns=columns))
    return recommend_df.reset_index(drop=True)

In [22]:
prediction = recommend(train_pivot, msd_basic_rating)
prediction

,pid,recommend-1,recommend-2,recommend-3,recommend-4,recommend-5,recommend-6,recommend-7,recommend-8,recommend-9,recommend-10
0,28,7520,2715,4796,2700,404,4214,5979,2698,2701,1333
1,42,3713,1828,1622,4420,4102,1581,1646,1826,1925,404
2,53,1159,4796,2715,1337,1925,7520,4753,1334,5691,3713
3,93,3713,1828,1925,327,2415,1826,1668,1581,6401,2704
4,95,7520,2698,2715,1159,1333,2700,5979,525,1874,5691
...,...,...,...,...,...,...,...,...,...,...,...
68,964,2698,4837,7520,2715,404,1337,5979,4796,1331,4811
69,971,2715,4796,525,1159,1335,1396,536,4811,1409,4638
70,977,1874,1826,1581,1590,4772,4811,1605,5691,4742,1925
71,980,1333,1159,2698,536,1396,5979,2770,1409,7520,2701


In [23]:
def evaluate_rprec(pred, real):
    evaluation = real.copy()
    evaluation['found'] = 0
    
    for index in real.index:
        row = real.loc[index]
        pid = row['pid']
        tid = row['tid']
        
        recommendation = pred.loc[pred['pid'] == pid, 'recommend-1':]
        if tid in recommendation.values[0]:
            evaluation.loc[index, 'found'] = 1
            
    rprec = np.mean(evaluation.groupby('pid')['found'].mean())
    return evaluation, rprec

In [24]:
evaluation, rprec = evaluate_rprec(prediction, test)
print('R-Precision: {}'.format(rprec))

R-Precision: 0.45114155251141547


In [25]:
cosine_playlist_matrix = cosine_similarity(train_pivot)

In [26]:
cosine_basic_rating = knn_basic(train_pivot, cosine_playlist_matrix)

In [27]:
prediction = recommend(train_pivot, cosine_basic_rating)
prediction

,pid,recommend-1,recommend-2,recommend-3,recommend-4,recommend-5,recommend-6,recommend-7,recommend-8,recommend-9,recommend-10
0,28,2715,7520,1333,2698,5979,4214,2700,1409,2701,4638
1,42,3713,5691,1622,1828,2705,1646,1396,2,5,1345
2,53,2715,524,4753,1333,4811,1159,7520,4796,4772,4638
3,93,1828,3713,2400,1826,1581,2364,3133,5691,4742,1925
4,95,2715,7520,2698,1333,525,4638,1159,2700,2364,5
...,...,...,...,...,...,...,...,...,...,...,...
68,964,2698,4837,1337,2715,404,7520,1925,4811,2704,2071
69,971,4811,2715,4796,525,1396,524,1159,1409,536,4638
70,977,1590,1826,1874,5691,2705,2715,1605,4742,1581,2165
71,980,1159,2701,1333,1396,2698,4811,1409,2715,4837,5979


In [28]:
evaluation, rprec = evaluate_rprec(prediction, test)
print('R-Precision: {}'.format(rprec))

R-Precision: 0.4595890410958904


In [82]:
def get_track_name(prediction, playlist_df, track_df):
    data = prediction.copy()
    data['pid'] = data['pid'].apply(lambda x: playlist_df.loc[playlist_df['pid'] == x, 'name'].values[0])
    
    for column in data.columns[1:]:
        data[column] = data[column].apply(lambda x: track_df.loc[track_df['tid'] == x, 'track_name'].values[0])
        
    return data

In [83]:
get_track_name(prediction, playlist_df, track_df)

,pid,recommend-1,recommend-2,recommend-3,recommend-4,recommend-5,recommend-6,recommend-7,recommend-8,recommend-9,recommend-10
0,Yeet,Congratulations,Swang,iSpy (feat. Lil Yachty),Bad and Boujee (feat. Lil Uzi Vert),White Iverson,No Role Modelz,Bounce Back,I'm the One,Caroline,Fake Love
1,Road Trippin',One Dance,Work,Work from Home,Don't Let Me Down,Panda,This Is What You Came For,Broccoli (feat. Lil Yachty),Crazy In Love,Yeah!,Stronger
2,NB,Congratulations,Drowning (feat. Kodak Black),Low Life,iSpy (feat. Lil Yachty),Chill Bill,XO TOUR Llif3,Swang,X (feat. Future),No Problem (feat. Lil Wayne & 2 Chainz),Fake Love
3,Catchy Songs,Don't Let Me Down,One Dance,Cold Water (feat. Justin Bieber & MØ),Never Be Like You,Roses,Trap Queen,Needed Me,Work,Antidote,Jumpman
4,roadtrippin,Congratulations,Swang,Bad and Boujee (feat. Lil Uzi Vert),iSpy (feat. Lil Yachty),goosebumps,Fake Love,XO TOUR Llif3,Bounce Back,Trap Queen,Yeah!
...,...,...,...,...,...,...,...,...,...,...,...
68,KILLA,Bad and Boujee (feat. Lil Uzi Vert),You Was Right,Mask Off,Congratulations,Redbone,Swang,Jumpman,Chill Bill,The Motto,Starboy
69,Rap it up,Chill Bill,Congratulations,X (feat. Future),goosebumps,Broccoli (feat. Lil Yachty),Drowning (feat. Kodak Black),XO TOUR Llif3,I'm the One,Go Flex,Fake Love
70,summer playlist,Sorry,Never Be Like You,Closer,Work,Panda,Congratulations,I Took A Pill In Ibiza - Seeb Remix,Antidote,Roses,Love Yourself
71,Gym,XO TOUR Llif3,Caroline,iSpy (feat. Lil Yachty),Broccoli (feat. Lil Yachty),Bad and Boujee (feat. Lil Uzi Vert),Chill Bill,I'm the One,Congratulations,You Was Right,White Iverson
